In [1]:
import mlagents
from mlagents_envs.environment import UnityEnvironment as UE
import numpy as np
from mlagents_envs.environment import ActionTuple


In [2]:
# env =  UE(file_name="stage0_160523\stage0_copy",seed=1,side_channels=[])
env =  UE(file_name="S1_noref\\build",seed=1,side_channels=[],worker_id=2,no_graphics = False)
env.reset()
behavior_names = list(env.behavior_specs.keys())
behavior_value = list(env.behavior_specs.values())
# for i in range(len(behavior_names)):
#     print(behavior_names[i])
#     print("obs:",behavior_value[i].observation_specs, "   act:", behavior_value[0].action_spec)
DecisionSteps, TerminalSteps = env.get_steps(behavior_names[0])
agentsNum = len(DecisionSteps.agent_id)
# print("exist:",DecisionSteps.agent_id,"   Dead:",TerminalSteps.agent_id)
# print("reward:",DecisionSteps.reward,"reward_dead:",TerminalSteps.reward)
# print("obs:",DecisionSteps.obs,"DeadObs:",TerminalSteps.obs)
# print("interrupted:", TerminalSteps.interrupted)


In [3]:
import torch
import torch.nn as nn
#(3,128,128) --> (64,7,7)
image = torch.tensor(DecisionSteps.obs[0]).reshape(1,3,128,128)
image.size()

torch.Size([1, 3, 128, 128])

In [4]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

vision_output_dim = 3136
num_words = 35  # Number of unique words in the vocabulary
language_output_dim = 128
embedding_dim = 128
mixing_dim = 256
lstm_hidden_dim = 256
num_actions = 4

# (3,128,128) --> (64,7,7) = 3136 (3-layer CNN)
class VisualModule(nn.Module): 
    def __init__(self):
        super(VisualModule, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=3, padding=0),
            nn.ReLU()
        )
        # self.conv = nn.Sequential(
        #     nn.Conv2d(3, 32, kernel_size=5, stride=2, padding=2),
        #     nn.ReLU(),
        #     nn.Conv2d(32, 64, kernel_size=5, stride=2, padding=2),
        #     nn.ReLU(),
        #     nn.Conv2d(64, 128, kernel_size=5, stride=2, padding=2),
        #     nn.ReLU(),
        #     nn.Conv2d(128, 64, kernel_size=5, stride=2, padding=1),
        #     nn.ReLU(),
        #     nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
        #     nn.ReLU(),
        #     nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
        #     nn.ReLU(),
        # )

    def forward(self, vt):
        encoded_vt = self.conv(vt)
        return encoded_vt.view(vt.size(0), -1).squeeze()

# one-hot encoding [0 0 1 0 0] --> 128 dimensional embedding (FF)
# S1:5 S2:5 S3:11 S4:9 --> 30 + 5 (noun) = 35 in total
class LanguageModule(nn.Module): 
    def __init__(self, num_words, embedding_dim):
        super(LanguageModule, self).__init__()
        self.embedding = nn.Linear(num_words, embedding_dim)

    def forward(self, lt):
        embedded_lt = self.embedding(lt)
        return embedded_lt

# 3136(vision) + 128 (language) --> 256 dimensional embedding (FF)
class MixingModule(nn.Module):
    def __init__(self, vision_output_dim, language_output_dim, mixing_dim):
        super(MixingModule, self).__init__()
        self.linear = nn.Linear(vision_output_dim + language_output_dim, mixing_dim)

    def forward(self, vision_output, language_output):
        combined_output = torch.cat((vision_output, language_output), dim=0)
        mixed_output = self.linear(combined_output)
        return mixed_output

class LSTMModule(nn.Module):
    def __init__(self,mixing_dim,lstm_hidden_dim):
        super(LSTMModule, self).__init__()
        self.lstm = nn.LSTMCell(mixing_dim, lstm_hidden_dim)
    
    def forward(self,mixed_output,lstm_hidden_state):
        lstm_hidden_state = self.lstm(mixed_output, lstm_hidden_state) 
        # lstm_output = lstm_hidden_state[0] # output is (hidden_state,cell_state), we need hidden state, shape (1,256)
        return lstm_hidden_state

class Agent(nn.Module):
    def __init__(self, num_words, embedding_dim, vision_output_dim, language_output_dim, mixing_dim, lstm_hidden_dim,num_actions):
        super(Agent, self).__init__()
        self.language_module = LanguageModule(num_words, embedding_dim)
        self.visual_module = VisualModule()
        self.mixing_module = MixingModule(vision_output_dim, language_output_dim, mixing_dim)
        self.lstm_module = LSTMModule(mixing_dim, lstm_hidden_dim)
        self.action_predictor = nn.Linear(lstm_hidden_dim, num_actions)
        self.value_estimator = nn.Linear(lstm_hidden_dim, 1)

    def forward(self, vt, lt, lstm_hidden_state):
        vision_output = self.visual_module(vt)
        language_output = self.language_module(lt)
        mixed_output = self.mixing_module(vision_output, language_output).unsqueeze(0)
        lstm_output = self.lstm_module(mixed_output,lstm_hidden_state)
        action_probs = self.action_predictor(lstm_output[0]) 
        value_estimate = self.value_estimator(lstm_output[0])
        return action_probs,value_estimate,lstm_output
        
        
    def save(self, episode, ALG_NAME, ENV_ID):
        path = os.path.join('model', '_'.join([ALG_NAME, ENV_ID]))
        if not os.path.exists(path):
            os.makedirs(path)
        torch.save(self.state_dict(), os.path.join(path, f'agent_{episode}.pt'))

    def load(self, episode, ALG_NAME, ENV_ID):
        path = os.path.join('model', '_'.join([ALG_NAME, ENV_ID]))
        self.load_state_dict(torch.load(os.path.join(path, f'agent_{episode}.pt')))    

        


In [5]:
visual_model = VisualModule()
vt = image
image_emb = visual_model(vt)
print(f'The shape of image embedding is {image_emb.size()}')

index = 5
language_model = LanguageModule(num_words,embedding_dim)
lt = torch.eye(num_words)[:, index]
language_emb = language_model(lt)
print(f'The shape of language embedding is {language_emb.size()}')

mixing_model = MixingModule(vision_output_dim,language_output_dim,mixing_dim)
mix_emb = mixing_model(image_emb,language_emb)
print(f'The shape of mix embedding is {mix_emb.size()}')

lstm = LSTMModule(mixing_dim,lstm_hidden_dim)
lstm_hidden_state = (torch.zeros(1, lstm_hidden_dim), torch.zeros(1, lstm_hidden_dim))
hidden_state = lstm(mix_emb.unsqueeze(0),lstm_hidden_state)
print(f'The shape of lstm hidden state is {hidden_state[0].size()}')

agent = Agent(num_words, embedding_dim, vision_output_dim, language_output_dim, mixing_dim, lstm_hidden_dim,num_actions)
action, value, lstm_hidden_state= agent(vt,lt,lstm_hidden_state)


The shape of image embedding is torch.Size([3136])
The shape of language embedding is torch.Size([128])
The shape of mix embedding is torch.Size([256])
The shape of lstm hidden state is torch.Size([1, 256])


In [12]:
import argparse
import time
import json
import matplotlib.pyplot as plt
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
# add arguments in command --train/test
# parser = argparse.ArgumentParser(description='Train or test neural net motor controller.')
# parser.add_argument('--train', dest='train', action='store_true', default=False)
# parser.add_argument('--test', dest='test', action='store_true', default=True)
# args = parser.parse_args()
train = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) 
S0_ALG_NAME = 'S0_without'
S0_ENV_ID = '2'
S0_episode = 5341
ALG_NAME = 'S1_without'
ENV_ID = '1'
TRAIN_EPISODES = 20000  # number of overall episodes for training
TEST_EPISODES = 10  # number of overall episodes for testing
MAX_STEPS = 500  # maximum time step in one episode
LAM = 0.95  # reward discount in TD error
lr = 0.0001  #0.00005 
speed = 1
num_steps = 100 # the step for updating the network

if __name__ == '__main__':
    agent = Agent(num_words, embedding_dim, vision_output_dim, language_output_dim, mixing_dim, lstm_hidden_dim,num_actions)
    agent.load(S0_episode,S0_ALG_NAME,S0_ENV_ID)
    agent.to(device)
    optimizer = optim.RMSprop(agent.parameters(), lr=lr)
    best_score = 0
    hashmap = {
    0: 'capsule',
    1: 'cube',
    2: 'cylinder',
    3: 'prism',
    4: 'sphere'}
    if train:
        entropy_term = 0
        all_episode_reward = []
        all_average_reward = []
        all_steps = []
        all_actor_loss = []
        all_critic_loss = []
        all_entropy_loss = []
        all_total_loss = []
        tracked_agent = -1
        for episode in range(TRAIN_EPISODES):
            t0 = time.time()
            episode_reward = 0
            # env.reset()
            behavior_name=list(env.behavior_specs)[0]
            spec=env.behavior_specs[behavior_name]
            # state = env.reset().astype(np.float32)
            STEPS = 0
            decision_steps, terminal_steps = env.get_steps(behavior_name)
            # state -- vt, lt, lstm
            vt = torch.tensor(decision_steps.obs[0]).reshape(1,3,128,128).to(device)
            index = int(decision_steps.obs[1])
            print(f'---{hashmap[index]}---')
            # 0-capsule,1-cube,2-cylinder,3-prism,4-sphere 
            lt = torch.eye(num_words)[:, index].to(device)
            lstm_hidden_state = (torch.zeros(1, lstm_hidden_dim).to(device), torch.zeros(1, lstm_hidden_dim).to(device))
            done = False
            while True:

                # Need to use when calculating the loss
                log_probs = []
                # values = []
                values = torch.empty(0).to(device)
                rewards = []

                for steps in range(num_steps):
                    lstm_hidden_state = tuple(tensor.detach() for tensor in lstm_hidden_state)
                    STEPS += 1
                    policy_dist, value, lstm_hidden_state = agent(vt,lt,lstm_hidden_state)
                    # value = value.detach()
                    dist = F.softmax(policy_dist.detach(),dim=1).cpu().numpy()
                    

                    action_dist = Categorical(F.softmax(policy_dist.detach(),dim=1))
                    # action_dist = Categorical(F.softmax(policy_dist,dim=1))
                    action = action_dist.sample() # sample an action from action_dist
                    action_onehot = F.one_hot(torch.tensor(action),num_actions).cpu()
                    
                    log_prob = torch.log(F.softmax(policy_dist,dim=1)[0][action])
                    # log_prob = torch.log(F.softmax(policy_dist,dim=1)[0][action])
                    # entropy = -np.sum(np.mean(dist)* np.log(dist))
                    entropy = F.cross_entropy(policy_dist.detach(), action)

                    discrete_actions = np.array(action_onehot).reshape(1,4)*speed
                    action_tuple = ActionTuple()
                    action_tuple.add_discrete(discrete_actions)
                    env.set_actions(behavior_name,action_tuple)
                    env.step()
                    decision_steps, terminal_steps = env.get_steps(behavior_name)

                    if tracked_agent == -1 and len(decision_steps) >= 1:
                        tracked_agent = decision_steps.agent_id[0]
                        # print(tracked_agent)

                    if tracked_agent in terminal_steps: # roll over or hit the target
                        print('Agent in terminal steps')
                        done = True
                        reward = terminal_steps[tracked_agent].reward
                        if reward > 0:
                            pass
                        else: reward = -1 # roll over or other unseen conditions

                        print(f'Terminal Step reward: {reward}')

                    elif tracked_agent in decision_steps: # the agent which requires action
                        reward = decision_steps[tracked_agent].reward
                        # print(f'Decision Step reward: {reward}')
                        if reward<0:
                            print(f'Decision Step reward: {reward}')
                    if STEPS >= MAX_STEPS:
                        reward = -10
                        print(f'Max Step Reward: {reward}')
                        env.reset()
                        done = True
                    if STEPS % 100 == 0:
                        print (f'Step: {STEPS}')

                    episode_reward = episode_reward + reward

                    rewards.append(reward)
                    # values.append(value)
                    values = torch.cat((values, value), dim=0)
                    log_probs.append(log_prob)
                    entropy_term = entropy_term + entropy
                    vt_new = torch.tensor(decision_steps.obs[0]).reshape(1,3,128,128).to(device)
                    vt = vt_new

                    if done or steps == num_steps-1:
                        # _, Qval,_ = agent(vt_new,lt,lstm_hidden_state)
                        # Qval = Qval.detach()
                        break
                
                
                discounted_rewards = np.zeros_like(values.cpu().detach().numpy())
                cumulative = 0
                for t in reversed(range(len(rewards))):
                    cumulative = rewards[t] + LAM * cumulative # Monte Carlo
                    discounted_rewards[t] = cumulative
                # print(f'rewards:{rewards}, discounted_rewards:{discounted_rewards}')
                # Advantage Actor Critic

                # Qvals[-1] = rewards[t] + LAM * Qval      or       Qvals[-1] = rewards[t]                   
                # for t in range(len(rewards)-1):
                #         Qvals[t] = rewards[t] + LAM * values[t+1]
                
                # r_(t+1) = R(s_t|a_t)--> reward[t]        a_t, V_t = agent(s_t)
                # A_t = r_(t+1) + LAM * V_(t+1) - V_t 
                #     = Q_t - V_t
                
                # Monte Carlo Advantage = reward + LAM * cumulative_reward
                # Actor_loss = -log(pai(s_t|a_t))*A_t
                # Critic_loss = A_t.pow(2) *0.5
                # Entropy_loss = -F.entropy(pai(St),index) * 0.001

                # entropy = -np.sum(np.mean(dist) * np.log(dist))
                
                #update actor critic
                
                # values = torch.FloatTensor(values).requires_grad_(True).to(device)
                discounted_rewards = torch.FloatTensor(discounted_rewards.astype(np.float32)).to(device)
                log_probs = torch.stack(log_probs)
                advantage = discounted_rewards - values
                actor_loss = (-log_probs * advantage).mean()
                critic_loss = 0.5 * torch.square(advantage).mean()
                entropy_term /= num_steps
                entropy_loss = -0.001 * entropy_term
                ac_loss = actor_loss + critic_loss + entropy_loss
                # ac_loss = values.mean()
                optimizer.zero_grad()
                ac_loss.backward()
                optimizer.step()
                # for name, param in agent.named_parameters():
                #     if param.grad is not None:
                #         print(name, param.grad)
                #     else:
                #         print(name, "gradients not computed")
                # for name, param in agent.named_parameters():
                #     if name == 'value_estimator.weight':
                #         print(name, param)
                
                
                if done: break


            all_episode_reward.append(episode_reward)
            all_steps.append(STEPS)
            all_actor_loss.append(actor_loss)
            all_critic_loss.append(critic_loss)
            all_entropy_loss.append(entropy_loss)
            all_total_loss.append(ac_loss)
            if episode >= 100:
                avg_score = np.mean(all_episode_reward[-100:])
                all_average_reward.append(avg_score)
                if avg_score > best_score:
                    best_score = avg_score
                    agent.save(episode, ALG_NAME, ENV_ID)
                    print(f'-----The best score for averaging previous 100 episode reward is {best_score}. Model has been saved-----')
                print('Training  | Episode: {}/{}  | Episode Reward: {:.0f}  | Average Reward {:.2f}  | Actor loss: {:.2f} | Critic loss: {:.2f} | Entropy loss: {:.4f}  | Total Loss: {:.2f} | Total Steps: {}' \
                    .format(episode + 1, TRAIN_EPISODES, episode_reward, avg_score, actor_loss, critic_loss,entropy_loss,  ac_loss, STEPS))
            else:  print('Training  | Episode: {}/{}  | Episode Reward: {:.0f}  | Actor loss: {:.2f} | Critic loss: {:.2f} | Entropy loss: {:.4f}  | Total Loss: {:.2f} | Total Steps: {}' \
                    .format(episode + 1, TRAIN_EPISODES, episode_reward, actor_loss, critic_loss, entropy_loss,  ac_loss, STEPS))
            if episode%500 == 0:
                    agent.save(episode, ALG_NAME, ENV_ID)
                    print("Model has been saved")
        print(all_average_reward)
        agent.save(episode ,ALG_NAME, ENV_ID)
        print("Model has been saved")

        data = {
                    'all_average_reward': all_average_reward,
                    'all_episode_reward': all_episode_reward,
                    'all_actor_loss': all_actor_loss,
                    'all_critic_loss': all_critic_loss,
                    'all_entropy_loss': all_entropy_loss,
                    'all_total_loss': all_total_loss,
                    'all_steps': all_steps,
                } 
        file_path = f'{ALG_NAME}_{ENV_ID}.txt'
        with open(file_path, 'w') as file:
            json.dump(data, file)



cuda
cylinder
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 1/20000  | Episode Reward: 10  | Actor loss: -0.01 | Critic loss: 0.62 | Entropy loss: -0.0000  | Total Loss: 0.62 | Total Steps: 7
Model has been saved
sphere
Decision Step reward: -1.0
Decision Step reward: -1.0


c:\Users\linzj\anaconda3\envs\rl\lib\site-packages\ipykernel_launcher.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Decision Step reward: -1.0
Decision Step reward: -1.0
Step: 100
Decision Step reward: -1.0
Decision Step reward: -1.0
Decision Step reward: -1.0
Decision Step reward: -1.0
Decision Step reward: -1.0
Step: 200
Decision Step reward: -1.0
Decision Step reward: -1.0
Decision Step reward: -1.0
Step: 300
Step: 400
Max Step Reward: -10
Step: 500
Training  | Episode: 2/20000  | Episode Reward: -22  | Actor loss: -0.53 | Critic loss: 7.85 | Entropy loss: -0.0002  | Total Loss: 7.32 | Total Steps: 500
sphere
Decision Step reward: -1.0
Decision Step reward: -1.0
Decision Step reward: -1.0
Step: 100
Decision Step reward: -1.0
Step: 200
Decision Step reward: -1.0
Decision Step reward: -1.0
Step: 300
Decision Step reward: -1.0
Decision Step reward: -1.0
Step: 400
Decision Step reward: -1.0
Decision Step reward: -1.0
Max Step Reward: -10
Step: 500
Training  | Episode: 3/20000  | Episode Reward: -20  | Actor loss: -1.00 | Critic loss: 6.85 | Entropy loss: -0.0004  | Total Loss: 5.85 | Total Steps: 500

In [ ]:
print(all_average_reward)
list_string = str(all_average_reward)
file_path = f"{ALG_NAME}_{ENV_ID}.txt"  # Specify the file path and name
with open(file_path, "w") as file:
    # Write the list string to the file
    file.write(list_string)

[-10.46, -10.22, -9.69, -9.25, -9.17, -9.17, -9.55, -9.33, -9.61, -10.03, -10.57, -10.27, -10.15, -10.19, -10.0, -10.12, -10.02, -10.06, -9.62, -9.94, -9.92, -10.11, -10.09, -10.14, -10.24, -10.29, -10.45, -10.15, -10.59, -10.82, -10.88, -11.2, -11.41, -11.25, -10.8, -10.32, -9.9, -10.14, -9.81, -9.81, -9.75, -9.59, -9.08, -8.95, -8.67, -8.56, -8.67, -8.37, -8.56, -8.38, -8.75, -8.28, -8.26, -8.21, -8.07, -8.14, -8.07, -8.17, -7.83, -7.48, -7.54, -7.74, -7.86, -8.18, -7.88, -8.38, -8.38, -8.31, -8.5, -8.61, -8.79, -9.02, -9.03, -9.02, -8.93, -9.01, -8.6, -8.6, -9.17, -9.27, -9.17, -8.88, -8.94, -8.91, -8.6, -8.59, -8.76, -8.69, -9.04, -9.11, -9.29, -9.42, -9.59, -9.99, -9.76, -9.73, -10.01, -10.16, -10.29, -10.18, -10.08, -9.8, -9.76, -9.88, -9.93, -10.35, -10.11, -10.31, -9.66, -9.18, -8.97, -9.01, -9.37, -8.97, -8.99, -8.93, -9.14, -9.01, -9.06, -8.77, -8.87, -9.04, -9.43, -9.4, -9.4, -9.36, -9.11, -9.67, -9.19, -8.62, -8.59, -8.66, -8.67, -8.61, -8.74, -8.75, -8.84, -8.49, -8.41, -7

In [ ]:
from torch.distributions import Categorical
device = torch.device("cpu")
episode = 5341
speed = 1
MAX_STEPS = 500
TEST_EPISODES = 100
ALG_NAME = 'S0_without'
ENV_ID = '2'
tracked_agent = -1
env.reset()
agent = Agent(num_words, embedding_dim, vision_output_dim, language_output_dim, mixing_dim, lstm_hidden_dim,num_actions)
agent.load(episode,ALG_NAME,ENV_ID)
average = 0
for episode in range(TEST_EPISODES):
            STEPS = 0
            episode_reward = 0
            # env.reset()
            behavior_name=list(env.behavior_specs)[0]
            spec=env.behavior_specs[behavior_name]
            # state = env.reset().astype(np.float32)

            # state -- vt, lt, lstm
            vt = torch.tensor(DecisionSteps.obs[0]).reshape(1,3,128,128).to(device)
            index = 5 #e.g
            lt = torch.eye(num_words)[:, index].to(device)
            lstm_hidden_state = (torch.zeros(1, lstm_hidden_dim).to(device), torch.zeros(1, lstm_hidden_dim).to(device))
            done = False
            while not done:
                STEPS += 1                
                lstm_hidden_state = tuple(tensor.detach() for tensor in lstm_hidden_state)
                policy_dist, value, lstm_hidden_state = agent(vt,lt,lstm_hidden_state)
                # value = value.detach()
                dist = F.softmax(policy_dist.detach(),dim=1).cpu().numpy()
                

                action_dist = Categorical(F.softmax(policy_dist.detach(),dim=1))
                # action_dist = Categorical(F.softmax(policy_dist,dim=1))
                action = action_dist.sample() # sample an action from action_dist
                action_onehot = F.one_hot(torch.tensor(action),num_actions).cpu()

                discrete_actions = np.array(action_onehot).reshape(1,4)*speed
                action_tuple = ActionTuple()
                action_tuple.add_discrete(discrete_actions)
                env.set_actions(behavior_name,action_tuple)
                env.step()
                decision_steps, terminal_steps = env.get_steps(behavior_name)

                if tracked_agent == -1 and len(decision_steps) >= 1:
                    tracked_agent = decision_steps.agent_id[0]
                    # print(tracked_agent)

                if tracked_agent in terminal_steps: # roll over or hit the target
                    print('Agent in terminal steps')
                    done = True
                    reward = terminal_steps[tracked_agent].reward
                    if reward > 0:
                        pass
                    else: reward = -1 # roll over or other unseen conditions

                    print(f'Terminal Step reward: {reward}')

                elif tracked_agent in decision_steps: # the agent which requires action
                    reward = decision_steps[tracked_agent].reward
                    # print(f'Decision Step reward: {reward}')
                    # if reward<0:
                    #     print(f'Decision Step reward: {reward}')

                if STEPS >= MAX_STEPS:
                        reward = -10
                        print(f'Max Step Reward: {reward}')
                        env.reset()
                        done = True

                episode_reward = episode_reward + reward
                vt_new = torch.tensor(decision_steps.obs[0]).reshape(1,3,128,128).to(device)
                vt = vt_new
            average += episode_reward / TEST_EPISODES
            print(f'Episode: {episode}, Episode reward: {episode_reward}')
print(f'Average Episode Reward: {average}')


c:\Users\linzj\anaconda3\envs\rl\lib\site-packages\ipykernel_launcher.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Agent in terminal steps
Terminal Step reward: 10.0
Episode: 0, Episode reward: 10.0
Agent in terminal steps
Terminal Step reward: 10.0
Episode: 1, Episode reward: 10.0
Agent in terminal steps
Terminal Step reward: 10.0
Episode: 2, Episode reward: 10.0
Agent in terminal steps
Terminal Step reward: 10.0
Episode: 3, Episode reward: 10.0
Agent in terminal steps
Terminal Step reward: 10.0
Episode: 4, Episode reward: 10.0
Agent in terminal steps
Terminal Step reward: 10.0
Episode: 5, Episode reward: 10.0
Agent in terminal steps
Terminal Step reward: 10.0
Episode: 6, Episode reward: 10.0
Agent in terminal steps
Terminal Step reward: 10.0
Episode: 7, Episode reward: 10.0
Agent in terminal steps
Terminal Step reward: 10.0
Episode: 8, Episode reward: 10.0
Agent in terminal steps
Terminal Step reward: 10.0
Episode: 9, Episode reward: 10.0
Agent in terminal steps
Terminal Step reward: 10.0
Episode: 10, Episode reward: 10.0
Agent in terminal steps
Terminal Step reward: 10.0
Episode: 11, Episode rew

In [ ]:
print(all_average_reward)

In [ ]:
import matplotlib.pyplot as plt


# Generate x-axis values as the indices of the list
x_axis = range(len(all_average_reward))

# Plotting the figure
plt.plot(x_axis, all_average_reward)
plt.xlabel('Episode')
plt.ylabel('Rewards')
plt.title('Rewards (Average 100 episodes) vs Episode')+
plt.show()

In [ ]:
episode = 738
env =  UE(file_name="280623_1\\build",seed=1,side_channels=[],worker_id=0,no_graphics = False)
env.reset()
behavior_names = list(env.behavior_specs.keys())
behavior_value = list(env.behavior_specs.values())
DecisionSteps, TerminalSteps = env.get_steps(behavior_names[0])
agent = Agent(num_words, embedding_dim, vision_output_dim, language_output_dim, mixing_dim, lstm_hidden_dim,num_actions)
agent.load(episode)


In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
# dist = np.array((2,1,2,2))
# entropy = -np.sum(np.mean(dist)* np.log(dist))
# entropy

action = torch.tensor([[0.1,0.2,0.1,1]])
index = torch.tensor([0])
print(action,index)
entropy_loss = F.cross_entropy(action, index)
entropy_loss

In [ ]:
# Do I need env.reset() at the beginning of each episode or unity file will reset it
# The utility of workstation GPU is only 1 percent (Memory Leak?) in laptop GPU 8 seconds / episode on average
# Can we make the unity environment accept action from GPU?
#  Model is still hard to learn meaningful actions -- Make sure the algorithm is correct (Weight Initialization? Experience Replay?) value layer how to update?

In [ ]:
smoothed_rewards = pd.Series.rolling(pd.Series(all_rewards), 10).mean()
smoothed_rewards = [elem for elem in smoothed_rewards]
plt.plot(all_rewards)
plt.plot(smoothed_rewards)
plt.plot()
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.show()

plt.plot(all_lengths)
plt.plot(average_lengths)
plt.xlabel('Episode')
plt.ylabel('Episode length')
plt.show()

In [ ]:
import argparse
import time
import matplotlib.pyplot as plt
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
# add arguments in command --train/test
# parser = argparse.ArgumentParser(description='Train or test neural net motor controller.')
# parser.add_argument('--train', dest='train', action='store_true', default=False)
# parser.add_argument('--test', dest='test', action='store_true', default=True)
# args = parser.parse_args()
train = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
#####################  hyper parameters  ####################

# ENV_ID = 'CartPole-v1'  # environment id
# RANDOM_SEED = 2  # random seed, can be either an int number or None
# RENDER = False  # render while training

# ALG_NAME = 'AC'
ALG_NAME = 'A2C'
ENV_ID = 'S0'
TRAIN_EPISODES = 200000  # number of overall episodes for training
TEST_EPISODES = 10  # number of overall episodes for testing
MAX_STEPS = 1200  # maximum time step in one episode
LAM = 0.95  # reward discount in TD error
lr = 0.001
speed = 2
# LR_A = 0.001  # learning rate for actor
# LR_C = 0.01  # learning rate for critic


###############################  Actor-Critic  ####################################


# class Actor(nn.Module):
#     def __init__(self, state_dim, action_num, lr=0.001):
#         super(Actor, self).__init__()
#         self.model = nn.Sequential(
#             nn.Linear(state_dim, 30),
#             nn.ReLU(),
#             nn.Linear(30, action_num)
#         )
#         self.optimizer = optim.Adam(self.parameters(), lr=lr)

#     def learn(self, state, action, td_error):
#         self.optimizer.zero_grad()
#         logits = self.model(torch.FloatTensor(state))
#         loss = td_error * torch.nn.functional.cross_entropy(logits, torch.LongTensor([action]))
#         loss.backward()
#         self.optimizer.step()
#         return loss.item()

#     def get_action(self, state, greedy=False):
#         logits = self.model(torch.FloatTensor(state))
#         probs = torch.nn.functional.softmax(logits, dim=-1).detach().numpy()
#         if greedy:
#             return np.argmax(probs)
#         return np.random.choice(len(probs[0]), p=probs[0])

#     def save(self):
#         path = os.path.join('model', '_'.join([ALG_NAME, ENV_ID]))
#         if not os.path.exists(path):
#             os.makedirs(path)
#         torch.save(self.state_dict(), os.path.join(path, 'model_actor.pt'))

#     def load(self):
#         path = os.path.join('model', '_'.join([ALG_NAME, ENV_ID]))
#         self.load_state_dict(torch.load(os.path.join(path, 'model_actor.pt')))


# class Critic(nn.Module):
#     def __init__(self, state_dim, lr=0.01):
#         super(Critic, self).__init__()
#         self.model = nn.Sequential(
#             nn.Linear(state_dim, 30),
#             nn.ReLU(),
#             nn.Linear(30, 1)
#         )
#         self.optimizer = optim.Adam(self.parameters(), lr=lr)

#     def learn(self, state, reward, state_, done):
#         self.optimizer.zero_grad()
#         d = 0 if done else 1
#         v_ = self.model(torch.FloatTensor(state_))
#         v = self.model(torch.FloatTensor(state))
#         td_error = reward + d * LAM * v_ - v
#         loss = td_error ** 2
#         loss.backward()
#         self.optimizer.step()
#         return td_error.item()

#     def save(self):
#         path = os.path.join('model', '_'.join([ALG_NAME, ENV_ID]))
#         if not os.path.exists(path):
#             os.makedirs(path)
#         torch.save(self.state_dict(), os.path.join(path, 'model_critic.pt'))

#     def load(self):
#         path = os.path.join('model', '_'.join([ALG_NAME, ENV_ID]))
#         self.load_state_dict(torch.load(os.path.join(path, 'model_critic.pt')))


if __name__ == '__main__':
    ''' 
    choose environment
    1. Openai gym:
    env = gym.make()
    2. DeepMind Control Suite:
    env = dm_control2gym.make()
    '''
    # env = gym.make(ENV_ID).unwrapped
    # # dm_control2gym.create_render_mode('example mode', show=True, return_pixel=False, height=240, width=320, camera_id=-1, overlays=(),
    # #              depth=False, scene_option=None)
    # # env = dm_control2gym.make(domain_name="cartpole", task_name="balance")

    # env.seed(RANDOM_SEED)  # reproducible
    # np.random.seed(RANDOM_SEED)
    # torch.manual_seed(RANDOM_SEED)  # reproducible

    # N_F = env.observation_space.shape[0]
    # N_A = env.action_space.n

    # print("observation dimension: %d" % N_F)  # 4
    # print("observation high: %s" % env.observation_space.high)  # [ 2.4 , inf , 0.41887902 , inf]
    # print("observation low : %s" % env.observation_space.low)  # [-2.4 , -inf , -0.41887902 , -inf]
    # print("num of actions: %d" % N_A)  # 2 : left or right

    # actor = Actor(state_dim=N_F, action_num=N_A, lr=LR_A)
    # # we need a good teacher, so the teacher should learn faster than the actor
    # critic = Critic(state_dim=N_F, lr=LR_C)

    agent = Agent(num_words, embedding_dim, vision_output_dim, language_output_dim, mixing_dim, lstm_hidden_dim,num_actions)
    agent.to(device)
    optimizer = optim.RMSprop(agent.parameters(), lr=lr)
    t0 = time.time()
    best_score = 0
    if train:
        all_episode_reward = []
        tracked_agent = -1
        for episode in range(TRAIN_EPISODES):
            print(f'Episode: {episode}')
            # state = env.reset().astype(np.float32)
            env.reset()
            behavior_name=list(env.behavior_specs)[0]
            spec=env.behavior_specs[behavior_name]

            # state -- vt, lt, lstm
            vt = torch.tensor(DecisionSteps.obs[0]).reshape(1,3,128,128).to(device)
            index = 5 #e.g
            lt = torch.eye(num_words)[:, index].to(device)
            step = 0  # number of step in this episode
            episode_reward = 0  # rewards of all steps
            lstm_hidden_state = (torch.zeros(1, lstm_hidden_dim).to(device), torch.zeros(1, lstm_hidden_dim).to(device))
            done = False
            while True:
                # if RENDER:
                #     env.render()

                # action = actor.get_action(state) state --> vt,lt
                
                action, value, lstm_hidden_state = agent(vt,lt,lstm_hidden_state) #action is the direct linear output from the model
                action_dist = Categorical(F.softmax(action,dim=1))
                index = action_dist.sample() # sample an action from action_dist
                action_onehot = F.one_hot(torch.tensor(index),num_actions).cpu()
                
                
                # action_onehot = F.one_hot(torch.argmax(action),num_actions)
                # print(action_onehot)

                # state_new, reward, done, info = env.step(action)
                # state_new = state_new.astype(np.float32)
                # if episode<50:
                #     if episode%2:
                #         action_onehot = torch.tensor([1,0,1,0])
                #     else: action_onehot = torch.tensor([1,0,0,1])                 
                # continuous_actions = np.empty((1, 0))
                discrete_actions = np.array(action_onehot).reshape(1,4)*speed #[forward, backward, right, left]
                
                # action_tuple = ActionTuple(continuous_actions,discrete_actions)
                action_tuple = ActionTuple()
                action_tuple.add_discrete(discrete_actions)
                env.set_actions(behavior_name,action_tuple)
                env.step()
                step += 1

                decision_steps, terminal_steps = env.get_steps(behavior_name)

                # print((vt == vt_new).all())

                if tracked_agent == -1 and len(decision_steps) >= 1:
                    tracked_agent = decision_steps.agent_id[0]
                    # print(tracked_agent)

                if tracked_agent in terminal_steps: # roll over or hit the target
                    print('Agent in terminal steps')
                    done = True
                    reward = terminal_steps[tracked_agent].reward
                    if reward > 0:
                        pass
                    else: reward = -1 # roll over or other unseen conditions

                    print(f'Terminal Step reward: {reward}')

                elif tracked_agent in decision_steps: # the agent which requires action
                    reward = decision_steps[tracked_agent].reward
                    # print(f'Decision Step reward: {reward}')
                    if reward<0:
                        print(f'Decision Step reward: {reward}')
                    vt_new = torch.tensor(decision_steps.obs[0]).reshape(1,3,128,128).to(device)
                    vt = vt_new
                

                if step >= MAX_STEPS:
                    reward = -1
                    print(f'Max Step Reward: {reward}')
                    done = True
                if step % 100 == 0:
                    print (f'Step: {step}')

                
                    
                episode_reward = episode_reward + reward


                

                # try:
                #     td_error = critic.learn(
                #         state, reward, state_new, done
                #     )  # learn Value-function : gradient = grad[r + lambda * V(s_new) - V(s)]
                #     actor.learn(state, action, td_error)  # learn Policy : true_gradient = grad[logPi(s, a) * td_error]                   
                try:
                    # d = 0 if done else 1
                    # v_ = self.model(torch.FloatTensor(state_))
                    # v = self.model(torch.FloatTensor(state))
                    if not done:
                        action_new, value_new, lstm_hidden_state_new = agent(vt_new,lt,lstm_hidden_state)

                        # initialize the value first
                        td_error = reward + LAM * value_new - value
                    else: td_error = reward - value


                    critic_loss = td_error ** 2

                    log_prob = action_dist.log_prob(index)
                    actor_loss = -(log_prob * td_error)

                    # logits = action # the actual model output (without softmax)
                    # labels = index  # one hot vector of the sampled action
                    print(action,index)
                    entropy_loss = F.cross_entropy(action, index)
                    print(entropy_loss)
                    total_loss = actor_loss + 0.5* critic_loss - 0.001*entropy_loss
                    # entropy loss entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logit, labels=tf.nn.softmax(logit))
                    # comb_loss = tf.reduce_mean((0.5 * value_loss + policy_loss - 0.01 * entropy))

                    total_loss = total_loss.to(device)

                    optimizer.zero_grad()
                    total_loss.backward(retain_graph=True)
                    print('hi')
                    optimizer.step()   
                    
              
                except KeyboardInterrupt:  # if Ctrl+C at running actor.learn(), then save model, or exit if not at actor.learn()
                    agent.save(episode)
                    print('model has been saved')

                # state = state_new
                # print(f'episode: {episode}, step:{step}')
                if done:
                    break
            
            if episode%500 == 0:
                agent.save(episode)
                print("Model has been saved")
                
            # all_episode_reward.append(episode_reward)
            # if episode >= 100:
            #     avg_score = np.mean(all_episode_reward[-100:])
            #     if avg_score > best_score:
            #         best_score = avg_score
            #         agent.save()
            #         print(f'The best score for averaging previous 100 episode reward is {best_score}. Model has been saved')



            # if episode == 0:
            #     all_episode_reward.append(episode_reward)
            # else:
            #     reference_value = all_episode_reward[-1] * 0.9 + episode_reward * 0.1
            #     if reference_value > all_episode_reward_best:
            #         agent.save()
            #         print(f'The best all_episode_reward is {reference_value}. Model has been saved')
            #         all_episode_reward_best = reference_value
            #     all_episode_reward.append(reference_value)

            print('Training  | Episode: {}/{}  | Episode Reward: {:.0f}  | Running Time: {:.4f}' \
                  .format(episode + 1, TRAIN_EPISODES, episode_reward, time.time() - t0))
            
            

            
            
        

        plt.plot(all_episode_reward)
        if not os.path.exists('image'):
            os.makedirs('image')
        plt.savefig(os.path.join('image', '_'.join([ALG_NAME, ENV_ID])))
        

    if not train:
        agent.load(episode)
        print("model has been loaded")
        tracked_agent = -1

        for episode in range(TEST_EPISODES):
            episode_time = time.time()
            # state = env.reset().astype(np.float32)
            vt = torch.tensor(DecisionSteps.obs[0]).reshape(1,3,128,128)
            index = 5 #e.g
            lt = torch.eye(num_words)[:, index]
            t = 0  # number of step in this episode
            episode_reward = 0
            while True:
                env.render()

                # action = actor.get_action(state, greedy=True)
                # state_new, reward, done, info = env.step(action)
                # state_new = state_new.astype(np.float32)
                # if done:
                #     reward = -20

                
                action, value, lstm_hidden_state = agent(vt,lt,lstm_hidden_state)
                action = F.one_hot(torch.argmax(action),num_actions)

                # state_new, reward, done, info = env.step(action)
                # state_new = state_new.astype(np.float32)
                continuous_actions = np.empty((1, 0))
                discrete_actions = np.array(action).reshape(1,4) #[forward, backward, right, left]
                action_tuple = ActionTuple(continuous_actions,discrete_actions)
                env.set_actions(behavior_name,action_tuple)
                env.step()
                decision_steps, terminal_steps = env.get_steps(behavior_name)
                vt_new = torch.tensor(decision_steps.obs).reshape(1,3,128,128)
                
                if tracked_agent == -1 and len(decision_steps) >= 1:
                    tracked_agent = decision_steps.agent_id[0]
                if tracked_agent in decision_steps:
                    # tracked_agent = decision_steps.agent_id[0]
                    reward = decision_steps[tracked_agent].reward 
                # if done:
                #     reward = -20  # reward shaping trick
                if tracked_agent in terminal_steps:
                    done = True
                    reward = -20

                episode_reward += reward
                vt = vt_new
                t += 1

                if done or t >= MAX_STEPS:
                    print('Testing  | Episode: {}/{}  | Episode Reward: {:.0f}  | Running Time: {:.4f}' \
                          .format(episode + 1, TEST_EPISODES, episode_reward, time.time() - t0))
                    break



In [ ]:
# Do we need to update the network every step?

In [ ]:
# Try to find the optimized MAX_STEP
# speed 1: 10000+ speed 2:1822 speed 3: 1918 speed 4: 900 speed 5: 951 speed 6:964 speed 7: 1181 
import random
import torch
import torch.nn.functional as F
# speed = 1
TRAIN_EPISODES = 20
tracked_agent = -1
num_actions = 4
average = 0
# for speed in range(30,70,5):
#     speed = speed/10
speed = 3
for episode in range(TRAIN_EPISODES):
    env.reset()
    behavior_name=list(env.behavior_specs)[0]
    step = 0
    while True:
        index = random.randint(0, 3) # sample an action from action_dist
        action_onehot = F.one_hot(torch.tensor(index),num_actions).cpu()
        discrete_actions = np.array(action_onehot).reshape(1,4)*speed #[forward, backward, right, left]
        action_tuple = ActionTuple()
        action_tuple.add_discrete(discrete_actions)
        env.set_actions(behavior_name,action_tuple)
        env.step()
        step += 1

        decision_steps, terminal_steps = env.get_steps(behavior_name)
        if tracked_agent == -1 and len(decision_steps) >= 1:
            tracked_agent = decision_steps.agent_id[0]
            
        if tracked_agent in terminal_steps: # roll over or hit the target
            reward = terminal_steps[tracked_agent].reward
            if reward > 0: # hit the target
                print(f'{episode}: {step} in total')
                average += step
                break
            else:           # roll over or other conditions
                env.reset()
                step = 0
                continue # roll over or other unseen conditions
        if tracked_agent in decision_steps: # the agent which requires action
            continue
average /= TRAIN_EPISODES
print(f'For speed {speed}, average random step for hitting the target is {average}')
      

In [ ]:
# the unity could not open in work station
# The agent in terminal step will also in decison step

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Sample data
object_sizes = [1, 2, 3, 4, 5]  # Object sizes on the x-axis
max_steps = [[4721, 2468, 1587, 1249, 1084],   # Max steps for each object size
             [4027, 1550, 738, 640, 505],
             [3320, 1391, 464, 331, 307],
             [3289, 882, 679, 424, 211],
             [3057, 1203, 470, 378, 222]]
speeds = [1, 2, 3, 4, 5]  # Discrete speeds for color-coding

# Color mapping for each speed
speed_color_mapping = {
    1: 'red',
    2: 'blue',
    3: 'green',
    4: 'orange',
    5: 'purple'
}

# Generate scatter plot
for i, size in enumerate(object_sizes):
    for j, steps in enumerate(max_steps[i]):
        speed = speeds[j]
        color = speed_color_mapping[speed]
        plt.scatter(size, steps, c=color)

# Set labels and title
plt.xlabel('Object Size')
plt.ylabel('Max Step')
plt.title('Scatter Plot')

plt.xlim(0, 6)
plt.xticks(np.arange(0, 7, 1))
plt.ylim(0, 5000)
plt.yticks(np.arange(0, 5001, 200))

# Create legend
legend_labels = [f'Speed {speed}' for speed, _ in speed_color_mapping.items()]
plt.legend(legend_labels, loc='upper right')

# Display the plot
plt.show()




In [ ]:
import torch
import torch.nn.functional as F
a = torch.tensor([[0.2460, 0.2503, 0.2647, 0.2389]])
b = F.softmax(a,dim=1)
b

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
import numpy as np

a = [1,2,3]
avg = np.mean(a[-10:])
avg